## Name Scope

**TensorFlow 不知道什么节点应该组合在一起，除非告诉他，那么告诉他的方法就是使用tf.name_scape(name)**

&emsp;&emsp;当神经网络的结构更加复杂、参数更多时，就需要一个更好的方式来传递和管理神经网络中的参数了。TensorFlow提供了通过变量名称（name）来创建或者获取一个变量的机制。通过这个机制，**在不同的函数中可以直接通过变量的名字来使用变量，而不需要将变量通过参数的形式到处传递。**

In [1]:
import tensorflow as tf

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with tf.name_scope("name_scope_test_1"):
    v1 = tf.get_variable (name = 'v', shape = [1], initializer = tf.constant_initializer(1.0))
    v2 = tf.Variable (tf.constant(2.0), name = 'v')
    v3 = tf.Variable (tf.constant(3.0), name = 'v')

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print ("v1 name is ", v1.name)
    print ("v2 name is ", v2.name)
    print ("v3 name is ", v3.name)

v1 name is  v:0
v2 name is  name_scope_test_1/v:0
v3 name is  name_scope_test_1/v_1:0


上面代码和输出结果表明：<br>
tf.name_scope()对tf.get_variable没有影响，但对tf.Variable有影响。 <br>
**不会在get_variable的声明的变量前加上空间名** <br>
**对于Variable所声明的变量，会在原有的name上加上命名空间，也就是name_scope_test/v:0** <br>
如果在tf.name_scope作用域下有相同的name，则会自动加上命名空间和会重命名name，即name_scope_test/v:0和name_scope_test/v_1:0

In [3]:
with tf.variable_scope ("variable_scope_test_1"):
    v1 = tf.get_variable (name = 'v', shape = [1], initializer = tf.constant_initializer (1.0))
    v2 = tf.Variable (tf.constant(2.0), name = 'v')
    v3 = tf.Variable (tf.constant(3.0), name = 'v')

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print ("v1 name is ", v1.name)
    print ("v2 name is ", v2.name)
    print ("v3 name is ", v3.name)

v1 name is  variable_scope_test_1/v:0
v2 name is  variable_scope_test_1/v_1:0
v3 name is  variable_scope_test_1/v_2:0


上面代码和输出结果表明： <br>
**tf.variable_scope对tf.get_variable有影响，会在原有的name上加上变量空间** <br>
也就是variable_scope_test/v:0。tf.Variable在tf.name_scope和tf.variable_scope作用域下的效果一样

In [4]:
"""
       如果在同一个命名空间下，使用get_variable声明两个名字相同的变量会报错，这也就是get_variable的一个优势
    防止变量重名以致的问题。
    ValueError: Variable variable_scope_test_2/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:
"""
with tf.variable_scope ("variable_scope_test_2"):
    v1 = tf.get_variable (name = 'v', shape = [1], initializer = tf.constant_initializer (1.0))
    v2 = tf.Variable (tf.constant(2.0), name = 'v')
    v3 = tf.Variable (tf.constant(3.0), name = 'v')
    v4 = tf.get_variable (name = 'v', shape = [1], initializer = tf.constant_initializer (1.0))

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print ("v1 name is ", v1.name)
    print ("v2 name is ", v2.name)
    print ("v3 name is ", v3.name)
    print ("v3 name is ", v4.name)

ValueError: Variable variable_scope_test_2/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "D:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1768, in __init__
    self._traceback = tf_stack.extract_stack()
  File "D:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3272, in create_op
    op_def=op_def)
  File "D:\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)


In [5]:
with tf.variable_scope ("variable_scope_test_3"):
    v1 = tf.get_variable (name = 'v', shape = [1], initializer = tf.constant_initializer (1.0))
    v2 = tf.Variable (tf.constant(2.0), name = 'v')
    v3 = tf.Variable (tf.constant(3.0), name = 'v')

with tf.variable_scope ("variable_scope_test_3", reuse = True):
    v4 = tf.get_variable('v', shape=[1])
    
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print ("v1 name is ", v1.name)
    print ("v2 name is ", v2.name)
    print ("v3 name is ", v3.name)
    print ("v4 name is ", v4.name)

v1 name is  variable_scope_test_3/v:0
v2 name is  variable_scope_test_3/v_1:0
v3 name is  variable_scope_test_3/v_2:0
v4 name is  variable_scope_test_3/v:0


In [7]:
"""
    当使用resue = True时声明新的变量的时候会报错
    ValueError: Variable variable_scope_test_4/v1 does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?
    这是因为因为设置reuse=True之后在该变量命名空间内，tf.get_variable只能获取已存在的变量而不能创建新变量
"""
with tf.variable_scope ("variable_scope_test_4"):
    v1 = tf.get_variable (name = 'v', shape = [1], initializer = tf.constant_initializer (1.0))
    v2 = tf.Variable (tf.constant(2.0), name = 'v')
    v3 = tf.Variable (tf.constant(3.0), name = 'v')

with tf.variable_scope ("variable_scope_test_4", reuse = True):
    v4 = tf.get_variable('v1', shape=[1])
    
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print ("v1 name is ", v1.name)
    print ("v2 name is ", v2.name)
    print ("v3 name is ", v3.name)
    print ("v4 name is ", v4.name)

ValueError: Variable variable_scope_test_4/v1 does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?

In [8]:
with tf.variable_scope('variable_scope_test'):
    v1 = tf.get_variable('v', shape=[1], initializer=tf.constant_initializer(1.0))
    v2 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')
    v3 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

with tf.variable_scope('variable_scope_test') as scope:
    v4 = tf.get_variable('v1', shape=[1], initializer=tf.constant_initializer(1.0))
    scope.reuse_variables()
    v5 = tf.get_variable('v', shape=[1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('the name of v1:', v1.name)
    print('the name of v2:', v2.name)
    print('the name of v3:', v3.name)
    print('the name of v4:', v4.name)
    print('the name of v5:', v5.name)

the name of v1: variable_scope_test/v:0
the name of v2: variable_scope_test/v_1:0
the name of v3: variable_scope_test/v_2:0
the name of v4: variable_scope_test/v1:0
the name of v5: variable_scope_test/v:0


**通过上述方法，可以声明也可以获取变量**

In [9]:
with tf.variable_scope('variable_scope_test_5'):
    v1 = tf.get_variable('v', shape=[1], initializer=tf.constant_initializer(1.0))
    v2 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')
    v3 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

with tf.variable_scope('variable_scope_test_5') as scope:
    v4 = tf.get_variable('v1', shape=[1], initializer=tf.constant_initializer(1.0))
    # 强调有重复变量
    scope.reuse_variables()
    v5 = tf.get_variable('v', shape=[1], initializer=tf.constant_initializer(11.0))

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('the name of v1:', v1.name)
    print ("the val of v1:", v1.eval())
    print('the name of v2:', v2.name)
    print('the name of v3:', v3.name)
    print('the name of v4:', v4.name)
    print('the name of v5:', v5.name)
    print ("the val of v1:", v1.eval())
    print ("the val of v5:", v5.eval())

the name of v1: variable_scope_test_5/v:0
the val of v1: [1.]
the name of v2: variable_scope_test_5_1/v:0
the name of v3: variable_scope_test_5_1/v_1:0
the name of v4: variable_scope_test_5/v1:0
the name of v5: variable_scope_test_5/v:0
the val of v1: [1.]
the val of v5: [1.]
